In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt

In [2]:
try:
    # needed only if you don't have the movies.txt file in your local or colab directory
    from google_drive_downloader import GoogleDriveDownloader as gdd
    file_id1='1iytA1n2z4go3uVCwE__vIKouTKyIDjEq'
    # movies.txt >> 9 GB data  ... below shared URL has the fileID to download
    #https://drive.google.com/file/d/1cRLjG6Pl4JEag-utmD-7nLipzxCdmkiQ/view?usp=sharing
    file_id2='1cRLjG6Pl4JEag-utmD-7nLipzxCdmkiQ'
    gdd.download_file_from_google_drive(file_id=file_id2,
                                        dest_path='data/movies.txt',
                                        unzip=True)
except:
    print('no need for laptop')

no need for laptop


In [3]:
start_time = datetime.datetime.now()
grp = gam.Graph_Amazon()
max_connected_gr_amazon_movies = grp.Create_Bipartite(file_name = 'data/movies.txt', \
                                                      n_movies = 40000, prs_out = 'dictionary')
end_time = datetime.datetime.now()
print('Calculation time-Crete graph from the movies file: {}'.format(end_time-start_time))

Calculation time-Crete graph from the movies file: 0:00:37.043212


In [4]:
bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)

In [5]:
# Data check
# some MOVIES don't start with B but instead full numbers like: 0790747324
print('max_connected_gr_amazon_movies:')
count_B = 0
for movie in top_nodes:
    if movie.startswith('B'):
        count_B += 1

print('Movies: {} ... starts with B: {}'.format(len(top_nodes), count_B))

count_A=0
for movie in bottom_nodes:
    if movie.startswith('A'):
        count_A += 1

print('Users: {} ... starts with A: {}'.format(len(bottom_nodes), count_A))

grp.Is_Connected_Bipartite(max_connected_gr_amazon_movies)
# edges between movies
#max_grp_top = bipartite.projected_graph(max_connected_gr_amazon_movies, grp.top_nodes)
# shows all the records BEFORE picking up the MAX CONNECTED component - most likely it will be a DISCONNECTED GRAPH
# returns a LIST
#edge_list_ALL = grp.Get_All_Edges_with_Weight(grp.gr_amazon_movies)
#print(edge_list_ALL[0:5])
#edge_list_connected = grp.Get_All_Edges_with_Weight(max_connected_gr_amazon_movies)
#list(max_connected_gr_amazon_movies.edges)[0:5]
# shows the nodes of the max connected component/subgraph
#grp.Show_Nodes()
#if your graph is too BIG, it is better not to draw it, it takes time..
#grp.Draw_Bipartite(max_connected_gr_amazon_movies)
#shows the edges with weights
#list(max_connected_gr_amazon_movies.edges.data('weight', default=1))[0:5]

max_connected_gr_amazon_movies:
Movies: 788 ... starts with B: 655
Users: 29294 ... starts with A: 29294
Is connected: True ... Is bipartite: True


In [6]:
#saves the movies/graph in the format of:
'''
A141HP4LYPW,B003AI2VGA,3.0
A141HP4LYPW,B000063W1R,4.0
A141HP4LYPW,6304286961,5.0
'''
#max_connected_gr_amazon_movies

##### TEST 
#file_name_saved = amp.Save_Movies(max_connected_gr_amazon_movies)

'\nA141HP4LYPW,B003AI2VGA,3.0\nA141HP4LYPW,B000063W1R,4.0\nA141HP4LYPW,6304286961,5.0\n'

In [7]:
# returns a list in the format of:
# [['userId', 'productId', 'score'], ['A141HP4LYPW', 'B003AI2VGA', '3.0']]
#the_giant_component_read = amp.Read_Connected_Movies("Movies_Connected_4643_11.5.2020_21.11.17.txt")

##### TEST 
#the_giant_component_read = amp.Read_Connected_Movies(file_name_saved)
#test[0].split(sep=",")

In [8]:
# it is mostly for a double check to see if the movies were saved to the file properly 
#and check if we read them and create a graph it will still be a connected graph or not

##### TEST 
#grp_giant = grp.Create_Graph_From_List_WITH_Weight(the_giant_component_read)

In [9]:
# double check if the numbers are matching before saving the records
# and after creating the graph from the saved file
print('BEFORE: # of nodes: {} ... # of edges: {}'.format(len(max_connected_gr_amazon_movies.nodes), len(max_connected_gr_amazon_movies.edges)))

##### TEST 
#print('AFTER: # of nodes: {} ... # of edges: {}'.format(len(grp_giant.nodes), len(grp_giant.edges)))

BEFORE: # of nodes: 30082 ... # of edges: 38233


In [10]:
# here is the check of the graph read

##### TEST 
#grp.Is_Connected_Bipartite(grp_giant)

In [11]:
# this is to fetch all nodes without edges, so that we can use them to predict the edges

##### TEST 
#grp_giant_no_edges = grp.Create_Graph_From_List_NO_EDGE(grp_giant)
#if you check it, you'll see no edges but nodes
#grp_giant_no_edges.edges
#list(grp_giant_no_weights.nodes)[0:5]
#grp.Is_Connected_Bipartite(grp_giant_no_weights)

In [12]:
def Normalize_Array(arr_to_normalize, axis=0):
    if axis == 1:
        arr_sum_per_row = arr_to_normalize.sum(axis=1)
        arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[0], 1)).T
    elif axis == 0:
        arr_sum_per_row = arr_to_normalize.sum(axis=0)
        arr_norm = arr_to_normalize/np.tile(arr_sum_per_row, (arr_sum_per_row.shape[1], 1)).T
    return arr_norm

# normalize P values (between 0 and 1) so that they can reflect probabilities 
# the probability with itself will be the highest
# !!! IMPORTANT !!!
# the probability with movie nodes will be ignored since we are looking for the relations(similarity-measure) 
# with other users
#P_norm = np.round(Normalize_Array(P), 6)

In [13]:
start_time = datetime.datetime.now()
P = nx.to_numpy_matrix(max_connected_gr_amazon_movies)
# by doing the below conversion, we are just considering the links between the nodes, we ignore the weights
# we try to understand if considering the weights will benefit us or not
#P[P > 0] = 1
P_norm = Normalize_Array(P)

#print(P)

# TEST - see if the matrix-P shows the correct values (to be sure if it's not changing the order)
# !!! IMPORTANT !!!
# apparently, it doesn't write the movies first and users last or doesn't follow any order..
node_0 = list(max_connected_gr_amazon_movies.nodes)[0]
node_1 = list(max_connected_gr_amazon_movies.nodes)[1]
edge_01 = max_connected_gr_amazon_movies.has_edge(node_0, node_1)
if edge_01 == True:
    weight_01 = max_connected_gr_amazon_movies.get_edge_data(node_0, node_1)['weight']

# Bxxxxxx = top_nodes = movies
print('node0={} .. node1={} .. edge01={} .. weight01={}'.format(node_0, node_1, edge_01, weight_01))
node_list = list(max_connected_gr_amazon_movies.nodes)
bottom_node_list = list(bottom_nodes)
top_node_list = list(top_nodes)
#print(node_list)
#print('\nbottom_nodes={}'.format(bottom_node_list))
#print('\ntop_nodes={}'.format(top_node_list))

n_nodes = P_norm.shape[0]
# initiate R matrix which includes the r values of each node and
# having the highest relation of the node by itself, setting the diagonal to 1
R = np.diag(np.ones(n_nodes), 0)
beta = 0.15
#R *= beta
R_zero = R.copy()
#print(P)
#print(P_norm)
#print(R)
#print(R_zero)
end_time = datetime.datetime.now()

n_reviews = len(max_connected_gr_amazon_movies.edges)
n_users = len(bottom_nodes)
n_movies = len(top_nodes)

print('Calculation time-P matrix generation from the graph: {}'.format(end_time-start_time))

node0=A141HP4LYPW .. node1=B003AI2VGA .. edge01=True .. weight01=3.0
Calculation time-P matrix generation from the graph: 0:01:24.508533


In [14]:
# Bxxxxxx = top_nodes = movies
node_check = node_list[0]
if node_check in top_node_list:
    print('this is a movie: {}'.format(node_check))
else:
    print('this is a user: {}'.format(node_check))

this is a user: A141HP4LYPW


In [15]:
list(max_connected_gr_amazon_movies.edges([node_list[0]]))
#max_connected_gr_amazon_movies.edges(['B003AI2VGA']) 
#nx.edges(max_connected_gr_amazon_movies, ['B003AI2VGA'])
#node_list.index('B003AI2VGA')

[('A141HP4LYPW', 'B003AI2VGA'),
 ('A141HP4LYPW', 'B000063W1R'),
 ('A141HP4LYPW', '6304286961'),
 ('A141HP4LYPW', '5556167281'),
 ('A141HP4LYPW', '6303998690'),
 ('A141HP4LYPW', 'B000AMWIVM'),
 ('A141HP4LYPW', 'B00008G1PX'),
 ('A141HP4LYPW', 'B002YJMMBA'),
 ('A141HP4LYPW', 'B000KKQNRO'),
 ('A141HP4LYPW', 'B000VULA9E'),
 ('A141HP4LYPW', 'B00004W221'),
 ('A141HP4LYPW', 'B000J0XJC2'),
 ('A141HP4LYPW', 'B00005V9I1'),
 ('A141HP4LYPW', 'B00015HX90')]

In [16]:
start_time = datetime.datetime.now()
user_indexes = []
movie_indexes = []
for nn in node_list:
    if nn in bottom_node_list:
        user_indexes.append(node_list.index(nn))
    else:
        movie_indexes.append(node_list.index(nn))

print('Calculation time-user-movie indexes: {}'.format(end_time-start_time))
end_time = datetime.datetime.now()

print('\n')
print(movie_indexes)
# users are usually too many, so will not print them
#print(user_indexes)


Calculation time-user-movie indexes: -1 day, 23:59:59.920511


[1, 9, 23, 48, 51, 61, 71, 379, 397, 404, 415, 600, 602, 1132, 1138, 1149, 1151, 1153, 1541, 1547, 1585, 1612, 1630, 1656, 1660, 1732, 1740, 1774, 1804, 1981, 1983, 2124, 2127, 2257, 2261, 2265, 2849, 2854, 2859, 2862, 2867, 2884, 2912, 2959, 3100, 3175, 3181, 3321, 3354, 3362, 3377, 3389, 3538, 3540, 3603, 3613, 4037, 4039, 4065, 4067, 4071, 4076, 4104, 4148, 4181, 4182, 4184, 4191, 4195, 4211, 4483, 4487, 4499, 4502, 4510, 4522, 4530, 4533, 4574, 4616, 4618, 4628, 4673, 4674, 4675, 4718, 4778, 4780, 4882, 4921, 4948, 5101, 5135, 5375, 5377, 5381, 5384, 5391, 5395, 5460, 5499, 5575, 5651, 5671, 5674, 5684, 5687, 5708, 5711, 5718, 5882, 5902, 5910, 5929, 5937, 5941, 5952, 5968, 5990, 6009, 6037, 6041, 6043, 6067, 6213, 6215, 6227, 6242, 6294, 6304, 6311, 6321, 6325, 6358, 6378, 6404, 6410, 6417, 6450, 6454, 6464, 6469, 6476, 6484, 6489, 6500, 6504, 6513, 6580, 6593, 6601, 6866, 6885, 6903, 6916, 6920, 6957, 6960, 6967, 6970

In [17]:
# SUM-check
#print('SUM-check')
#P_norm.sum(axis=1)

In [18]:
#savetxt(amp.FileNameUnique(prefix = "P_norm_", suffix = '.csv'), P_norm, delimiter=',')

In [19]:
#df_movies = amp.Reorganize_Edges_DataFrame(max_connected_gr_amazon_movies, input_type = 'Graph')

In [20]:
# with 40K records selected, 
# MEMORY of python hits to 45GB at some point and works around 34GB in general
# CPU works at around 390%
# there should be a better way to do this...
# tried cupy but it fails in Mac (latest OS version doesn't have support)
# Colab shows like importing cupy but then during the execution it fails at some point, couldn't fix it... 
# need to retry in a Windows laptop if cupy works or not..
# but even if it works, the number users reviewing more than 1 movie is too few
start_time = datetime.datetime.now()
n_steps = 3

for i in range(n_steps):
    R = (1-beta)*np.dot(P_norm, R) + beta*R_zero
    print('step-{} completed .. date: {}'.format(i, datetime.datetime.now()))
    
#print(R)
end_time = datetime.datetime.now()
print('Calculation time-steps walked: {}'.format(end_time-start_time))

step-0 completed .. date: 2020-05-12 16:52:19.606747
step-1 completed .. date: 2020-05-12 16:58:44.020069
step-2 completed .. date: 2020-05-12 17:05:26.121967
Calculation time-steps walked: 0:19:41.178833


In [72]:
# user similarity check
# fetch the values for node=0 (which is a user)
ref_user_idx = 3
# we will pick the most similar 10 users in this case
# this value should be picked as around the below ratio and I believe, 
# if any of the top-similar users reviewed a movie, 
# we can guess like our user will watch the movie: WILL BE TESTED
# OR we can pick a high number (>ratio) for the top similarity and the percentage>10 maybe counted as the candidate
#top_similarity = n_reviews/n_movies
top_similarity = 30
user_test = np.array(list(R[ref_user_idx]))
print('R matrix values for the user (r vector of the user): \n{}'.format(user_test))

# sort the values of each column from smaller to the bigger
sorted_nodes = np.argsort(user_test)

user_similarity = []

# we will not consider the user himself or movies as similar nodes
for idx in sorted_nodes[0][0]:
    if idx != ref_user_idx and idx not in movie_indexes:
        user_similarity.append(idx)
        
user_similarity_top = user_similarity[-top_similarity:]

# ALL is too many, so we will not print it
#print('Ordered similarities-ALL: {}'.format(user_similarity))
print('Top similarities: {}'.format(user_similarity_top))

#for usr in user_similarity_top:
#    print(list(max_connected_gr_amazon_movies.nodes)[user_similarity_top[usr]])
#print(list(max_connected_gr_amazon_movies.nodes)[user_similarity_top[0]])

user_similarity_top_score = []
for usr_top in user_similarity_top:
    user_similarity_top_score.append(round(R[ref_user_idx, usr_top], 4))
    
print(user_similarity_top_score)

R matrix values for the user (r vector of the user): 
[[[0.00211035 0.04508148 0.00238493 ... 0.         0.         0.        ]]]
Top similarities: [11817, 23806, 23807, 23808, 23791, 23786, 1089, 188, 5066, 3155, 1911, 363, 1935, 195, 4492, 1488, 23795, 332, 783, 25829, 9301, 1668, 489, 652, 7, 6, 4, 0, 5, 2]
[0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0003, 0.0005, 0.0007, 0.0014, 0.0021, 0.0021, 0.0021, 0.0024]


In [73]:
# top_nodes = MOVIES, bottom_nodes = users
user_reviewed = []
user_reviewed_id = []
movie_list_ref = []
movie_reviewed = []
movie_reviewed_id = []
is_reviewed = []
movie_score = []
review_count_list = []

movie_by_reference_user = list(max_connected_gr_amazon_movies.edges([node_list[ref_user_idx]]))

for edge_m in movie_by_reference_user:
    # if the second item is a user then we will add the first item to the movie_list
    if edge_m[1].startswith('A'):
        movie_list_ref.append(edge_m[0])
    else:
        movie_list_ref.append(edge_m[1])

for u_idx in user_similarity_top:
    user_name = node_list[u_idx]
    for m in movie_list_ref:
        m_idx = node_list.index(m)
        is_reviewed.append(max_connected_gr_amazon_movies.has_edge(user_name, m))
        user_reviewed.append(user_name)
        user_reviewed_id.append(u_idx)
        movie_reviewed.append(m)
        movie_reviewed_id.append(m_idx)
        try:
            scr = max_connected_gr_amazon_movies.get_edge_data(user_name, m)['weight']
        except:
            scr = '...'  # NA - no review score
        #scr = str(max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], m)['weight'])
        #max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'B003AI2VGA')['weight']
        movie_score.append(scr)
        review_count = len(list(max_connected_gr_amazon_movies.edges([m])))
        review_count_list.append(review_count)
        
        #is_reviewed = max_connected_gr_amazon_movies.get_edge_data(u, m)['weight']


dict_summary = {'user': user_reviewed, 'movie': movie_reviewed, 
                'user_id': user_reviewed_id, 'movie_id': movie_reviewed_id, 
                'is_reviewed': is_reviewed, 'score': movie_score,
               'n_review': review_count_list}

'''
dict_summary = {'user': user_reviewed, 'movie': movie_reviewed, 
                'user_id': user_reviewed_id, 'movie_id': movie_reviewed_id, 
                'is_reviewed': is_reviewed}
'''

df_summary = pd.DataFrame(dict_summary)

In [23]:
#print(movie_by_reference_user)

In [24]:
'''
true_values=len(df_summary.query("movie=='B003AI2VGA' and is_reviewed==True"))
total_values=len(df_summary.query("movie=='B003AI2VGA'"))

# review ratio by similar users
ratio_similar = true_values/total_values*100
print(ratio_similar)
'''

'\ntrue_values=len(df_summary.query("movie==\'B003AI2VGA\' and is_reviewed==True"))\ntotal_values=len(df_summary.query("movie==\'B003AI2VGA\'"))\n\n# review ratio by similar users\nratio_similar = true_values/total_values*100\nprint(ratio_similar)\n'

In [38]:
# DOUBLE CHECK if edge calculation is correct
# The problem is many of the users just review 1 movie or very few movies
# total review count per user. How many movies did a specific user reviewed?

'''
user_review_counts = []
user_review_counts_ALL = 0
# total review count per movie. How many users did a specific movie reviewed by?
movie_review_counts = []
movie_review_counts_ALL = 0

for usr in bottom_node_list:
    usr_count = len(list(max_connected_gr_amazon_movies.edges([usr])))
    user_review_counts.append([usr, usr_count])
    user_review_counts_ALL += usr_count
    
for mov in top_node_list:
    mov_count = len(list(max_connected_gr_amazon_movies.edges([mov])))
    user_review_counts.append([mov, mov_count])
    movie_review_counts_ALL += mov_count
    
print(user_review_counts)
'''

In [74]:
ratio_review_list = []
review_count_list2 = []

for m in movie_list_ref:
    true_values = len(df_summary.query("movie=='" + m + "' and is_reviewed==True"))
    total_values = len(df_summary.query("movie=='" + m + "'"))
    review_count = len(list(max_connected_gr_amazon_movies.edges([m])))
    # review ratio by similar users
    ratio_similar = true_values/total_values*100
    ratio_review_list.append(ratio_similar)
    review_count_list2.append(review_count)

#dict_summary_ratio = {'movie': movie_list_ref, 'ratio_similar': ratio_review_list}
dict_summary_ratio = {'user-movie': movie_by_reference_user, 'ratio_similar': ratio_review_list,
                     'review_count': review_count_list2}

df_summary_ratio = pd.DataFrame(dict_summary_ratio)
print('# of users in the evaluation: {}'.format(top_similarity))
print('Total # of MOVIEs: {} USERS: {} REVIEWs: {},  in the graph'.format(n_movies, n_users, n_reviews))
print('\n')
print(df_summary_ratio)

# of users in the evaluation: 30
Total # of MOVIEs: 788 USERS: 29294 REVIEWs: 38233,  in the graph


                  user-movie  ratio_similar  review_count
0  (A1I7QGUDP04, B003AI2VGA)      20.000000             7
1  (A1I7QGUDP04, B0016OLXN2)      23.333333           188
2  (A1I7QGUDP04, B001QB5SCM)      23.333333           141
3  (A1I7QGUDP04, B000KKQNRO)      30.000000           607
4  (A1I7QGUDP04, B008X1O7KM)       3.333333           205
5  (A1I7QGUDP04, B000NOIVT0)      53.333333           111
6  (A1I7QGUDP04, B0006HBZXS)      30.000000           122
7  (A1I7QGUDP04, B001TGV882)      23.333333           117


In [28]:
len(list(max_connected_gr_amazon_movies.edges(['B0006N2DS8'])))

10

In [29]:
max_connected_gr_amazon_movies.edges(['B0006N2DS8'])

EdgeDataView([('B0006N2DS8', 'A328S9RN3U5'), ('B0006N2DS8', 'AVQ6TZYX1V4'), ('B0006N2DS8', 'A312U8J4NSK'), ('B0006N2DS8', 'ANGNKTK2J7J'), ('B0006N2DS8', 'A29FFT26RF6'), ('B0006N2DS8', 'A1VP89BT4LL'), ('B0006N2DS8', 'A3AZ4XMN4DB'), ('B0006N2DS8', 'A22GAQ5R9EE'), ('B0006N2DS8', 'A1PVM3V959J'), ('B0006N2DS8', 'A1D4NUM28WI')])

In [30]:
# returns the top-close users reviewed the below movie
df_summary.query("movie=='B008JAG4GW' and is_reviewed==True")

,user,movie,user_id,movie_id,is_reviewed,score,n_review
308,A16CZRQL23N,B008JAG4GW,489,20807,True,4.0,17


In [31]:
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 300)
print(df_summary)

            user       movie  user_id  movie_id  is_reviewed score  n_review
0    A35ZK3M8L9J  B003AI2VGA      652         1        False   ...         7
1    A35ZK3M8L9J  B000063W1R      652        71        False   ...       309
2    A35ZK3M8L9J  B0001G6PZC      652       602         True   5.0       539
3    A35ZK3M8L9J  B00022VM5I      652      3100         True   5.0        98
4    A35ZK3M8L9J  B001MBE63I      652      3362        False   ...        17
..           ...         ...      ...       ...          ...   ...       ...
595  A3V0T37BKLK  B0001WTUZI    29578     23966        False   ...        13
596  A3V0T37BKLK  B008UO5U9E    29578     24128        False   ...       133
597  A3V0T37BKLK  B001TGV882    29578     28225        False   ...       117
598  A3V0T37BKLK  B00112S8RS    29578     28775        False   ...       151
599  A3V0T37BKLK  B000AYEL9M    29578     29577         True   5.0         5

[600 rows x 7 columns]


In [32]:
## TEST
u_idx = 0
#weight_tst = max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'A141HP4LYPW')['weight']
weight_tst = max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'B003AI2VGA')['weight']
print(weight_tst)
list(max_connected_gr_amazon_movies.edges([node_list[u_idx]]))
print(node_list[9])

3.0
B00006HAXW


In [33]:
type(max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'B003AI2VGA'))

dict

In [34]:
type(max_connected_gr_amazon_movies)

networkx.classes.graph.Graph

In [35]:
u_idx = 9
m='B003AI2VGA'
#max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], 'A141HP4LYPW')['weight']
type(max_connected_gr_amazon_movies.get_edge_data(node_list[u_idx], m)['weight'])

TypeError: 'NoneType' object is not subscriptable